In [3]:
import sys
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import shapely
from shapely.geometry import Polygon, Point, shape, box, mapping, MultiPolygon
from shapely.affinity import translate
from shapely.ops import transform
from pyproj import Transformer
from datetime import datetime, timedelta
import glob
import importlib.util
import osgeo as gdal
import rasterio

sys.path.insert(1, "src")
from putils import save_svg, get_observation, plot_geometry, plot_matrix, validate_geom
from futils import forward_pass_farsite, forward_pass_farsite_24h, cleanup_farsite_outputs
from dautils import sample_winddirection, sample_windspeed, adjusted_state_EnKF_farsite, geom_to_state, state_to_geom, align_states, interpolate_geom, interpolate_geoms

# Daily perimeter updates for data assimilation

In [12]:
PERIMETERS_PATH = 'outputs/border2_perimeters.geojson'
perimeters_gdf = gpd.read_file(PERIMETERS_PATH)


# ---- helper: parse ArcGIS-style datetimes (often epoch-ms or ISO strings) ----
def to_utc_ts(x):
    if pd.isna(x):
        return pd.NaT
    # epoch milliseconds (common in ESRI exports)
    if isinstance(x, (int, float)) and x > 1e11:
        return pd.to_datetime(int(x), unit="ms", utc=True)
    # epoch seconds (less common)
    if isinstance(x, (int, float)) and x > 1e9:
        return pd.to_datetime(int(x), unit="s", utc=True)
    # strings / datelike
    return pd.to_datetime(x, utc=True, errors="coerce")



# candidate “edit/update” time fields (best → worst)
time_candidates = [
    "attr_ModifiedOnDateTime_dt",
    "poly_CreateDate",
    "attr_CreatedOnDateTime_dt",
    "poly_PolygonDateTime",
    "poly_DateCurrent",
]


# pick the first column that exists and has any non-null values
chosen_time_col = None
for c in time_candidates:
    if c in perimeters_gdf.columns and perimeters_gdf[c].notna().any():
        chosen_time_col = c
        break


if chosen_time_col is None:
    raise ValueError(f"No usable time column found among: {time_candidates}")

perimeters_gdf["_update_ts"] = perimeters_gdf[chosen_time_col].map(to_utc_ts)



# geometry area tie-breaker (works even if timestamps tie or are messy)
# (ensure projected CRS if you want true area; otherwise it's just a monotonic proxy)
perimeters_gdf["_area"] = perimeters_gdf.geometry.area



# Final sort:
# 1) update timestamp (ascending)
# 2) area (ascending; usually grows over time)
# 3) OBJECTID as stable deterministic tie-breaker if present
sort_cols = ["_update_ts", "_area"] + (["OBJECTID"] if "OBJECTID" in perimeters_gdf.columns else [])
gdf_sorted = perimeters_gdf.sort_values(sort_cols, ascending=True).reset_index(drop=True)

print("Sorted using:", chosen_time_col)

Sorted using: attr_ModifiedOnDateTime_dt


In [11]:
# Write sorted perimeters to GeoJSON
OUTPUT_PATH = "outputs/border2_perimeters_sorted.geojson"

# Optional but recommended: drop helper columns you added
cols_to_drop = [c for c in ["_update_ts", "_area"] if c in gdf_sorted.columns]
gdf_out = gdf_sorted.drop(columns=cols_to_drop)

# Write to GeoJSON
gdf_out.to_file(
    OUTPUT_PATH,
    driver="GeoJSON"
)

print(f"Wrote {len(gdf_out)} features to {OUTPUT_PATH}")

Wrote 4 features to outputs/border2_perimeters_sorted.geojson
